**Abubaker Saeed  20200798**

**Tonaja Mohamed 20200900**

**Ahmed Essam 20200853**

**Isaac Kenyi 20180433**

In [2]:
#Impoer The needed libraries
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
import pandas as pd
from pandas import read_csv
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.optimizers import RMSprop,Adam
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM
from sklearn.model_selection import train_test_split
import re

In [3]:
#load the dataset
data = pd.read_csv('amazon_reviews.csv')

In [4]:
data.shape

(17340, 4)

In [5]:
data.head()

,sentiments,cleaned_review,cleaned_review_length,review_score
0,positive,i wish would have gotten one earlier love it a...,19,5.0
1,neutral,i ve learned this lesson again open the packag...,88,1.0
2,neutral,it is so slow and lags find better option,9,2.0
3,neutral,roller ball stopped working within months of m...,12,1.0
4,neutral,i like the color and size but it few days out ...,21,1.0


In [6]:
#check if there is null or missing values
data.isnull()
data['cleaned_review']=data['cleaned_review'].apply(str)

In [ ]:
X = data.drop(['sentiments'], axis=1)
y = data['sentiments']
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)

In [ ]:
X

,cleaned_review,cleaned_review_length,review_score
0,i wish would have gotten one earlier love it a...,19,5.0
1,i ve learned this lesson again open the packag...,88,1.0
2,it is so slow and lags find better option,9,2.0
3,roller ball stopped working within months of m...,12,1.0
4,i like the color and size but it few days out ...,21,1.0
...,...,...,...
17335,i love this speaker and love can take it anywh...,30,5.0
17336,i use it in my house easy to connect and loud ...,13,4.0
17337,the bass is good and the battery is amazing mu...,41,5.0
17338,love it,2,5.0


In [ ]:
# Cleaning the texts
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
corpus = []
for i in range(0, 10000):
    review = re.sub('[^a-zA-Z]', ' ', X['cleaned_review'][i])
    review = review.lower()
    review = review.split()
    ps = PorterStemmer()
    review = [ps.stem(word) for word in review if not word in set(stopwords.words('english'))]
    review = ' '.join(review)
    corpus.append(review)

In [ ]:
corpus=pd.DataFrame(corpus, columns=['cleaned_review'])
corpus.head()

,cleaned_review
0,wish would gotten one earlier love make work l...
1,learn lesson open packag use product right awa...
2,slow lag find better option
3,roller ball stop work within month minim use p...
4,like color size day return period hold charg


In [ ]:
result=corpus.join(data[['review_score']])
result.head()

,cleaned_review,review_score
0,wish would gotten one earlier love make work l...,5.0
1,learn lesson open packag use product right awa...,1.0
2,slow lag find better option,2.0
3,roller ball stop work within month minim use p...,1.0
4,like color size day return period hold charg,1.0


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()
tfidf.fit(result['cleaned_review'])

TfidfVectorizer()

In [ ]:
X = tfidf.transform(result['cleaned_review'])
result['cleaned_review'][1]

'learn lesson open packag use product right away order mous august travel mous pack away bag visit famili pnw sinc mid septemb mous took charg work fine coupl week recharg mous work day two ceas function optic light flash turn noth cours return window well past'

In [ ]:
result.dropna(inplace=True)
result[result['review_score'] != 3]
result['Positivity'] = np.where(result['review_score'] > 3, 1, 0)
cols = [ 'review_score']
result.drop(cols, axis=1, inplace=True)
result.head()

,cleaned_review,Positivity
0,wish would gotten one earlier love make work l...,1
1,learn lesson open packag use product right awa...,0
2,slow lag find better option,0
3,roller ball stop work within month minim use p...,0
4,like color size day return period hold charg,0


In [ ]:
result.groupby('Positivity').size()


Positivity
0    4814
1    5186
dtype: int64

In [ ]:
X = result.cleaned_review
y = result.Positivity
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 0)

In [ ]:
max_fatures = 30000
tokenizer = Tokenizer(nb_words=max_fatures, split=' ')
tokenizer.fit_on_texts(result['cleaned_review'].values)
X1 = tokenizer.texts_to_sequences(result['cleaned_review'].values)
X1 = pad_sequences(X1)

In [ ]:
Y1 = pd.get_dummies(result['Positivity']).values
X1_train, X1_test, Y1_train, Y1_test = train_test_split(X1,Y1, random_state = 42)
print(X1_train.shape,Y1_train.shape)
print(X1_test.shape,Y1_test.shape)

(7500, 223) (7500, 2)
(2500, 223) (2500, 2)


In [ ]:
#First model RNN
model_RNN = keras.Sequential()
model_RNN.add(layers.SimpleRNN(64, activation="relu", input_shape=(X1.shape[1], 1)))
model_RNN.add(Dense(32, activation='relu'))
model_RNN.add(layers.Dense(1))

In [ ]:
#Second model LSTM
model_LSTM = keras.Sequential()
model_LSTM.add(layers.LSTM(64, input_shape=(X1.shape[1],1)))
model_LSTM.add(Dense(32, activation='relu'))
model_LSTM.add(layers.Dense(1))

In [ ]:
# compile the model
opt=RMSprop(learning_rate=0.01)
model_RNN.compile(optimizer=opt, loss='mse')

# early stopping callback
es = EarlyStopping(monitor='val_loss',
                   mode='min',
                   patience=50,
                   restore_best_weights = True)

# fit the model
history = model_RNN.fit(X1_train, Y1_train,
                    validation_data = (X1_test, Y1_test),
                    callbacks=[es],
                    epochs=50,
                    batch_size=50,
                    verbose=1)

Epoch 1/50
150/150 [==============================] - 8s 51ms/step - loss: 194411.8906 - val_loss: 11.6446
Epoch 2/50
150/150 [==============================] - 14s 93ms/step - loss: 69.2345 - val_loss: 0.2691
Epoch 3/50
150/150 [==============================] - 6s 41ms/step - loss: 0.2511 - val_loss: 0.2500
Epoch 4/50
150/150 [==============================] - 7s 50ms/step - loss: 0.2528 - val_loss: 0.2500
Epoch 5/50
150/150 [==============================] - 5s 35ms/step - loss: 0.2500 - val_loss: 0.2500
Epoch 6/50
150/150 [==============================] - 7s 46ms/step - loss: 0.2500 - val_loss: 0.2500
Epoch 7/50
150/150 [==============================] - 6s 37ms/step - loss: 0.2500 - val_loss: 0.2500
Epoch 8/50
150/150 [==============================] - 6s 40ms/step - loss: 0.2500 - val_loss: 0.2500
Epoch 9/50
150/150 [==============================] - 7s 44ms/step - loss: 0.2500 - val_loss: 0.2500
Epoch 10/50
150/150 [==============================] - 5s 35ms/step - loss: 0.2500 

In [ ]:
# compile the model
opt=RMSprop(learning_rate=0.01)
model_LSTM.compile(optimizer=opt, loss='mse')

# early stopping callback
es = EarlyStopping(monitor='val_loss',
                   mode='min',
                   patience=50,
                   restore_best_weights = True)

# fit the model
history = model_LSTM.fit(X1_train, Y1_train,
                    validation_data = (X1_test, Y1_test),
                    callbacks=[es],
                    epochs=50,
                    batch_size=50,
                    verbose=1)

Epoch 1/50
150/150 [==============================] - 28s 151ms/step - loss: 0.2875 - val_loss: 0.2548
Epoch 2/50
150/150 [==============================] - 18s 119ms/step - loss: 0.2514 - val_loss: 0.2510
Epoch 3/50
150/150 [==============================] - 18s 118ms/step - loss: 0.2509 - val_loss: 0.2506
Epoch 4/50
150/150 [==============================] - 17s 116ms/step - loss: 0.2507 - val_loss: 0.2507
Epoch 5/50
150/150 [==============================] - 20s 131ms/step - loss: 0.2506 - val_loss: 0.2505
Epoch 6/50
150/150 [==============================] - 17s 115ms/step - loss: 0.2506 - val_loss: 0.2505
Epoch 7/50
150/150 [==============================] - 17s 116ms/step - loss: 0.2505 - val_loss: 0.2504
Epoch 8/50
150/150 [==============================] - 18s 118ms/step - loss: 0.2504 - val_loss: 0.2504
Epoch 9/50
150/150 [==============================] - 18s 122ms/step - loss: 0.2504 - val_loss: 0.2503
Epoch 10/50
150/150 [==============================] - 17s 116ms/step - l

In [ ]:
# predict_review function
def predict_review(model, tokenizer, review, max_length):
    review = clean_text(review)
    seq = tokenizer.texts_to_sequences([review])
    pad_seq = pad_sequences(seq, maxlen=max_length, padding='post')
    pred = model.predict(pad_seq)
    sentiment = np.argmax(pred)
    sentiment_mapping = {0: 'Negative', 1: 'Neutral', 2: 'Positive'}
    return sentiment_mapping[sentiment]

In [ ]:
# Allow the user to input a new review and predict the result.
new_review = input("Enter a review: ")
rnn_prediction = predict_review(model_RNN, tokenizer, new_review, max_length)
lstm_prediction = predict_review(model_LSTM, tokenizer, new_review, max_length)
print(f"SimpleRNN Prediction: {rnn_prediction}")
print(f"LSTM Prediction: {lstm_prediction}")